In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



In [ ]:

df=pd.read_csv('/kaggle/input/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2021-03-31.csv')
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
def missing(df):
    total=df.isnull().sum()
    percent=df.isnull().sum()/len(df)
    return pd.concat([total,percent],axis=1,keys=['total','percent'])


In [ ]:
missing(df)

In [ ]:
df['Timestamp']=pd.to_datetime(df['Timestamp'], unit='s')

In [ ]:
df=df.dropna().reset_index(drop=True)

In [ ]:

import matplotlib.pyplot as plt

In [ ]:
df = df.set_index('Timestamp')

In [ ]:
df_day=df.resample('D').mean()
df_month=df.resample('M').mean()
df_year=df.resample('Y').mean()
df_quarter=df.resample('Q').mean()

In [ ]:
fig = plt.figure(figsize=[15, 4])
plt.subplot(141)
plt.plot(df_day.Weighted_Price, '-', label='By Days')
plt.legend()

plt.subplot(142)
plt.plot(df_month.Weighted_Price, '-', label='By Month')
plt.legend()

plt.subplot(143)
plt.plot(df_year.Weighted_Price, '-', label='By Year')
plt.legend()

plt.subplot(144)
plt.plot(df_quarter.Weighted_Price, '-', label='By Quarter')
plt.legend()

In [ ]:
from scipy import stats
import statsmodels.api as sm

In [ ]:
plt.figure(figsize=[80,80])
sm.tsa.seasonal_decompose(df_month.Weighted_Price).plot()
plt.show()

In [ ]:
print('test:p=%f'% sm.tsa.stattools.adfuller(df_month.Weighted_Price)[1])

In [ ]:
print('test:p=%f'% sm.tsa.stattools.adfuller(df_month.Weighted_Price)[1])

In [ ]:
df_month.Weighted_Price = np.log(df_month.Weighted_Price)

In [ ]:
print('test:p=%f'% sm.tsa.stattools.adfuller(df_month.Weighted_Price)[1])

In [ ]:

plt.plot(df_month.Weighted_Price, '-', label='By Month')
plt.legend()

In [ ]:
df_month.Weighted_Price =df_month.Weighted_Price.diff()

In [ ]:
df_month = df_month.drop(df_month.index[0])

In [ ]:
print('test:p=%f'% sm.tsa.stattools.adfuller(df_month.Weighted_Price)[1])

In [ ]:
plt.plot(df_month.Weighted_Price, '-', label='By Month')
plt.legend()

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

In [ ]:
model = SARIMAX(df_month.Weighted_Price, order=(3, 1, 3), seasonal_order=(2, 1, 2, 4))

In [ ]:
res = model.fit()

In [ ]:
res.summary()

In [ ]:
df_month['sarimax'] = res.fittedvalues

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(df_month.Weighted_Price)
plt.plot(df_month['sarimax'])
plt.show()

In [ ]:
forecast = res.predict(start=len(df_month), end=len(df_month) + 8)

In [ ]:
forecast = df_month['sarimax'].append(forecast)

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(df_month.Weighted_Price)
plt.plot(forecast)
plt.show()

In [ ]:
from itertools import product
p = range(0, 2, 1)
d = 1
q = range(0, 2, 1)
ps = range(0, 2, 1)
ds = 1
qs = range(0, 2, 1)
s = 4
params = list(product(p, q, ps, qs))
params

In [ ]:
results = []
for order in params:
    try:
        model = SARIMAX(df_month.Weighted_Price, order=(order[0], d, order[1]), seasonal_order=(order[2], ds, order[3], s))
        res = model.fit()
        results.append([order, res.aic])
    except:
        continue

In [ ]:
r = pd.DataFrame(results)
r.columns = ('order', 'AIC')
r.sort_values(by='AIC', ascending=True)

In [ ]:
model = SARIMAX(df_month.Weighted_Price, order=(1, 1, 1), seasonal_order=(0, 1, 0, 4))
res = model.fit()
res.summary()

In [ ]:
df_month['sarimax'][:5] = np.NaN
forecast = res.predict(start=len(df_month), end=len(df_month) + 8)

In [ ]:
forecast = df_month['sarimax'].append(forecast)

In [ ]:
plt.figure(figsize=(20, 12))
plt.plot(df_month.Weighted_Price)
plt.plot(forecast)
plt.show()